$\textbf{EM Algorithm for Mixture of Two Gaussian Distributions} $\
Input: $X = (x_{1}, x_{2}, ..., x_{N})$ is $N$-samples and $K$ is the number of Gaussian Distributions \
Output: MLE of $(X, parameters = (\mu, \sigma, \gamma))$ and parameters themselves.

Algorithm: \
1. Randomly initializing $\mu, \sigma, \gamma$ with the length of $K$ and normalizing $\gamma$ to make it sum add up to 1. \
2. Expectation step --> calculating weighted sums for $i=[1, 2, ..., N], k=[1, 2, ..., K]$
$$p_{ik} = \frac{\gamma_{k}N(x_{i}|\mu_{k}, \sigma_{k})}{\sum_{l=1}^{K} \gamma_{l}N(x_{i}|\mu_{l}, \sigma_{l})}$$ 
3. Maximization step --> updating the parameters for $k=[1, 2, ..., K]$
$$\mu_{k} = \frac{\sum_{i=1}^{N} p_{ik}x_{i}}{\sum_{i=1}^{N} p_{ik}}$$
$$\sigma_{k} = \frac{\sum_{i=1}^{N} p_{ik}(x_{i}-\mu_{k})^2}{\sum_{i=1}^{N} p_{ik}}$$
$$\gamma_{k} = \frac{1}{N}\sum_{i=1}^{N} p_{ik}$$
4. Apply above (2), (3) repeatedly until observing the convergency of MLE by comparing previous and current values.


Notes:
1. I have also made helper functions to calculate the pdf of normal distributions and MLE in order to avoid very long lines of codes in main function.
2. I used stopping value and compared pre and new states to conclude convergency, but we can also add max_iters (defaultly = 100) which will run the code with this depth. I added necessary code for that as well. Even though the estimated parameters will not be affected too much, one can also try to use max_iters version commented out in below codes.





In [1]:
# Importing numpy
import numpy as np

In [2]:
# Let's first define some function to calculate the pdf of Gaussian
def normal_pdf(x, mu, sigma):
    return (1.0 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-((x - mu) ** 2) / (2 * (sigma ** 2)))

# ML-estimate for X=(x1, x2, ..., xn) and K-dim mean, variance, and weights
def MLE(X, mu, sigma, gamma):
    K = len(mu)
    N = len(X)
    MLEi = 0
    MLE = 1
    for i in range(N):
        for k in range(K):
            MLEi += gamma[k] * normal_pdf(X[i], mu[k], sigma[k])
        MLE *= MLEi
        MLEi = 0
    return MLE

# Let's solve the problem
def EM_algorithm(X, K, stopping_criteria = 1e-6, max_iters = 100):
    # Initialization
    N = len(X)
    mu = np.random.rand(K)
    sigma = np.random.rand(K)
    gamma = np.random.rand(K)
    gamma = gamma / gamma.sum() # They are the weights and sum up to one (1)
    # EM algorithm
    MLE_pre = np.inf
    MLE_new = MLE(X, mu, sigma, gamma)
    while abs(MLE_pre - MLE_new) > stopping_criteria: # We can choose when to stop, not exactly zero but convergent to prev state
    #for i in range(max_iters): # With max_iters --> another version to define termination
        MLE_pre = MLE_new
        # Expectation-step
        p = np.zeros((N, K))
        for i in range(N):
            for k in range(K):
                p[i][k] = gamma[k] * normal_pdf(X[i], mu[k], sigma[k])
            p[i] = p[i] / p[i].sum()
        # Maximization-step
        for k in range(K):
            pk = p[:, k].sum()
            mu[k] = (p[:, k] * X).sum() / pk
            sigma[k] = np.sqrt((p[:, k] * ((X - mu[k]) ** 2)).sum() / pk)
            gamma[k] = pk / N
        MLE_new = MLE(X, mu, sigma, gamma)
    mle = MLE_new
    params = (mu, sigma, gamma)
    return mle, params


In [3]:
# To check the validity
np.random.seed(1) # For reproducibility 
x = np.random.rand(100)
print(EM_algorithm(x, 10))


(6120106.7433827, (array([0.02645614, 0.55682318, 0.1739329 , 0.30038531, 0.91624781,
       0.09945558, 0.41922496, 0.20463366, 0.14010361, 0.70795703]), array([0.01894344, 0.03711165, 0.00984908, 0.03607811, 0.04026135,
       0.00936325, 0.01566649, 0.00565022, 0.00572324, 0.05991644]), array([0.08020811, 0.11348062, 0.03051238, 0.12211877, 0.16630179,
       0.05992969, 0.11839452, 0.02850993, 0.0595889 , 0.2209553 ])))
